### Информация
- **Автор**: Зитцер Данил
- **Дата создания**: 4 мая 2021 года, 11:34:13
- **Краткое описание**: Скрипт распознавания фигур будет работать через RestAPI, соответственно изображения ему будут приходить в JSON формате. Есть два варианта как numpy.ndarray перегонять в JSON:
    1. Воспользоваться Pydantic
    2. Написать JSON Encoder для numpy
- **Полезные ссылки**:
    - [Pydantic models](https://pydantic-docs.helpmanual.io/usage/models/)
    - [Support Generic Container Types #380](https://github.com/samuelcolvin/pydantic/issues/380)
    - [Numpy JSON Encoder](https://stackoverflow.com/questions/26646362/numpy-array-is-not-json-serializable)
- **Выводы**: При использовании Pydantic, получается cлишком много кода. Ведь всё, что изначально надо было сделать - это np.array().tolist()

In [ ]:
import json
import numpy as np
import pydantic

from nptyping import NDArray
from pydantic import BaseModel, validator
from typing import Any, Union, List

### Pydantic model

#### First variant 

In [ ]:
class _ArrayMeta(type):
    def __getitem__(self, t):
        return type('Array', (Array,), {'__dtype__': t})


class Array(np.ndarray, metaclass=_ArrayMeta):
    @classmethod
    def __get_validators__(cls):
        yield cls.validate_type

    @classmethod
    def validate_type(cls, val):
        dtype = getattr(cls, '__dtype__', None)
        if isinstance(dtype, tuple):
            dtype, shape = dtype
        else:
            shape = tuple()

        result = np.array(val, dtype=dtype, copy=False, ndmin=len(shape))
        assert not shape or len(shape) == len(result.shape)  # ndmin guarantees this

        if any((shape[i] != -1 and shape[i] != result.shape[i]) for i in range(len(shape))):
            result = result.reshape(shape)
        return result

#### Second variant 

In [ ]:
class NPImage(BaseModel):
    image: Union[List, np.ndarray]
        
    @validator('image')
    def image_validator(cls, v):
        if isinstance(v, list):
            v = np.array(v, dtype=np.uint8)
        if not isinstance(v, np.ndarray):
            raise TypeError('must be a numpy.ndarray')
        if v.dtype != np.uint8:
            raise TypeError('numpy.ndarray must have dtype equal numpy.uint8')
        if len(v.shape) != 3:
            raise ValueError('numpy.ndarray must have 3 dimension')
        if v.shape[2] != 3:
            raise ValueError('numpy.ndarray must have 3 channels')
        return v
        
    class Config:
        arbitrary_types_allowed = True
        
        json_encoders = {
            np.ndarray: lambda x: x.tolist()
        }

In [ ]:
# npimage = NPImage(image=np.ones(shape=(3, 3, 3), dtype=np.uint8))
npimage = NPImage(image=np.ones(shape=(3, 3, 3), dtype=np.uint8))
npimage.image.shape

In [ ]:
npimage_raw = npimage.json()
npimage_raw

In [ ]:
NPImage.parse_raw(npimage_raw)

### Numpy JSON encoder 

In [ ]:
class NumpyJsonEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [ ]:
image = np.zeros(shape=(3, 3, 3), dtype=np.uint8)

In [ ]:
json.dumps(image, cls=NumpyJsonEncoder)

### Numpy request to FastAPI

In [ ]:
import requests

In [ ]:
nparray = np.arange(3)

In [ ]:
json.dumps({'image': nparray.tolist()})

In [ ]:
r = requests.post('http://127.0.0.1:5000/images/', data=json.dumps({'image': nparray.tolist()}))
r.status_code

In [ ]:
r.json()

In [ ]:
r = requests.post('http://127.0.0.1:5000/images/', data=NPImage(image=np.ones(shape=(320, 320, 3), dtype=np.uint8)).json())
r.status_code

In [ ]:
r.json()